In [1]:
#dependencies
import pandas as pd #used to bring in zone/zip csv
import geopandas #package to do all geographic analysis/cleaning
import matplotlib.pyplot as plt #used to plot graphs in notebook
from shapely.geometry import MultiPoint, Polygon, Point #used to create certain shapes
import math

In [131]:
# file with all zip codes to use
file = 'ny_new_york_zip_codes_geo.min.json'

In [132]:
# import geoJSON file
gdf = geopandas.read_file(file)
gdf

,STATEFP10,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,geometry
0,36,12205,3612205,B5,G6350,S,40906445,243508,+42.7187855,-073.8292399,N,"POLYGON ((-73.87052 42.75123, -73.86861 42.756..."
1,36,12009,3612009,B5,G6350,S,135241924,2168637,+42.6975663,-074.0355422,N,"POLYGON ((-74.10891 42.65300, -74.10889 42.653..."
2,36,14804,3614804,B5,G6350,S,144718714,232123,+42.3172588,-077.8479358,N,"POLYGON ((-77.92747 42.34775, -77.92632 42.347..."
3,36,14836,3614836,B5,G6350,S,77612958,131305,+42.5429182,-077.8781933,N,"MULTIPOLYGON (((-77.95599 42.47433, -77.95600 ..."
4,36,14536,3614536,B5,G6350,S,47193482,425175,+42.5439751,-078.0836709,N,"POLYGON ((-78.05030 42.53850, -78.05024 42.538..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1789,36,10503,3610503,B5,G6350,S,38497,0,+41.0265555,-073.8753095,N,"POLYGON ((-73.87383 41.02557, -73.87631 41.025..."
1790,36,10535,3610535,B5,G6350,S,1169775,154348,+41.3351693,-073.7939193,N,"POLYGON ((-73.81299 41.33241, -73.81285 41.332..."
1791,36,14569,3614569,B5,G6350,S,197585027,1112470,+42.7346931,-078.1685932,N,"POLYGON ((-78.19740 42.82658, -78.19619 42.826..."
1792,36,14478,3614478,B5,G6350,S,30311215,0,+42.5778234,-077.1263221,N,"POLYGON ((-77.09451 42.62465, -77.09428 42.624..."


In [133]:
# import file with Zones and Zip Codes to merge to gdf (to summarize by zone)
zip_df = pd.read_csv('nyp_zips.csv')
zip_df['zip'] = zip_df['zip'].astype(str) #convert zips to objext/string as stored in geoJSON/gdf
zip_df

,zone,zip
0,East Harlem,10029
1,East Harlem,10035
2,Gramercy Park,10010
3,Gramercy Park,10016
4,Gramercy Park,10017
...,...,...
199,Brooklyn,11236
200,Brooklyn,11237
201,Brooklyn,11238
202,Brooklyn,11239


In [134]:
# add zone column to gdf
gdf = gdf.merge(zip_df, left_on ='ZCTA5CE10', right_on ='zip', how ='left')
gdf.head()

,STATEFP10,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,geometry,zone,zip
0,36,12205,3612205,B5,G6350,S,40906445,243508,+42.7187855,-073.8292399,N,"POLYGON ((-73.87052 42.75123, -73.86861 42.756...",NaN,NaN
1,36,12009,3612009,B5,G6350,S,135241924,2168637,+42.6975663,-074.0355422,N,"POLYGON ((-74.10891 42.65300, -74.10889 42.653...",NaN,NaN
2,36,14804,3614804,B5,G6350,S,144718714,232123,+42.3172588,-077.8479358,N,"POLYGON ((-77.92747 42.34775, -77.92632 42.347...",NaN,NaN
3,36,14836,3614836,B5,G6350,S,77612958,131305,+42.5429182,-077.8781933,N,"MULTIPOLYGON (((-77.95599 42.47433, -77.95600 ...",NaN,NaN
4,36,14536,3614536,B5,G6350,S,47193482,425175,+42.5439751,-078.0836709,N,"POLYGON ((-78.05030 42.53850, -78.05024 42.538...",NaN,NaN


In [135]:
# dissolve technique will group zones into a single geometric shape
world = gdf[['zone', 'geometry']]
zones = gdf.dissolve(by='zone')
zones.to_file("zones_file.geojson", driver="GeoJSON")
zones.head()

,geometry,STATEFP10,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,zip
zone,,,,,,,,,,,,,
Brooklyn,"POLYGON ((-73.95985 40.57401, -73.96037 40.574...",36,11234,3611234,B5,G6350,S,19547849,2143727,+40.6050798,-073.9117206,N,11234
Central Harlem,"MULTIPOLYGON (((-73.94879 40.79748, -73.94910 ...",36,10030,3610030,B5,G6350,S,722465,0,+40.8182666,-073.9428564,N,10030
Chelsea and Clinton,"POLYGON ((-74.00273 40.74029, -74.00464 40.741...",36,10011,3610011,B5,G6350,S,1705243,0,+40.7420017,-074.0005940,N,10011
East Harlem,"MULTIPOLYGON (((-73.92869 40.78118, -73.93149 ...",36,10029,3610029,B5,G6350,S,2136945,0,+40.7916981,-073.9438757,N,10029
Gramercy Park,"POLYGON ((-73.98451 40.73696, -73.98445 40.737...",36,10017,3610017,B5,G6350,S,829152,0,+40.7523468,-073.9724169,N,10017


## Scalable Method

In [136]:
# Create dictionary of dictionaries for each zone that returns the "exterior" and "buffered exterior" 20 coordinates
zone_names = pd.unique(zip_df['zone'])
keys = range(len(zone_names))
r = {}
for x in keys:
    r[x] = {} #creates sub-dictionary
    r[x]['Name'] = zone_names[x] 
    r[x]['Group'] = zones.loc[[zone_names[x]], 'geometry']
    if r[x]['Group'].geom_type[0] == 'MultiPolygon': #second [0] filters to the Polygon level
        r[x]['Exterior'] = int(math.ceil(len(r[x]['Group'][0][0].exterior.coords)/20)) #returns every nth number to use below
        r[x]['Exterior_Coords'] = r[x]['Group'][0][0].exterior.coords[0::r[x]['Exterior']] #returns every nth coordinate (20 total)
        r[x]['Buffer'] = int(math.ceil(len(r[x]['Group'][0][0].buffer(0.01,16, join_style = 2).exterior.coords)/20))
        r[x]['Buffer_Coords'] = r[x]['Group'][0][0].buffer(0.01,16, join_style = 2).exterior.coords[0::r[x]['Buffer']]    
    else: 
        r[x]['Exterior'] = int(math.ceil(len(r[x]['Group'][0].exterior.coords)/20)) 
        r[x]['Exterior_Coords'] = r[x]['Group'][0].exterior.coords[0::r[x]['Exterior']]
        r[x]['Buffer'] = int(math.ceil(len(r[x]['Group'][0].buffer(0.01,16, join_style = 2).exterior.coords)/20))
        r[x]['Buffer_Coords'] = r[x]['Group'][0].buffer(0.01,16, join_style = 2).exterior.coords[0::r[x]['Buffer']] 
r
    

/var/folders/4_/q_7tb_w52611r6nz11nb05qm0000gp/T/ipykernel_2922/870032046.py:10: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  r[x]['Exterior'] = int(math.ceil(len(r[x]['Group'][0][0].exterior.coords)/20)) #returns every nth number to use below
/var/folders/4_/q_7tb_w52611r6nz11nb05qm0000gp/T/ipykernel_2922/870032046.py:11: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  r[x]['Exterior_Coords'] = r[x]['Group'][0][0].exterior.coords[0::r[x]['Exterior']] #returns every nth coordinate (20 total)
/var/folders/4_/q_7tb_w52611r6nz11nb05qm0000gp/T/ipykernel_2922/870032046.py:12: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. 

/var/folders/4_/q_7tb_w52611r6nz11nb05qm0000gp/T/ipykernel_2922/870032046.py:13: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  r[x]['Buffer_Coords'] = r[x]['Group'][0][0].buffer(0.01,16, join_style = 2).exterior.coords[0::r[x]['Buffer']]
/var/folders/4_/q_7tb_w52611r6nz11nb05qm0000gp/T/ipykernel_2922/870032046.py:10: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  r[x]['Exterior'] = int(math.ceil(len(r[x]['Group'][0][0].exterior.coords)/20)) #returns every nth number to use below
/var/folders/4_/q_7tb_w52611r6nz11nb05qm0000gp/T/ipykernel_2922/870032046.py:11: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geom

{0: {'Name': 'East Harlem',
  'Group': zone
  East Harlem    MULTIPOLYGON (((-73.92869 40.78118, -73.93149 ...
  Name: geometry, dtype: geometry,
  'Exterior': 3,
  'Exterior_Coords': [(-73.928686, 40.781184),
   (-73.935929, 40.783807),
   (-73.93551, 40.7859),
   (-73.932714, 40.78929),
   (-73.931404, 40.791146),
   (-73.925754, 40.791303),
   (-73.926622, 40.79163),
   (-73.92654, 40.79452),
   (-73.926879, 40.799987),
   (-73.918837, 40.798814),
   (-73.914684, 40.795799),
   (-73.915592, 40.791998),
   (-73.923803, 40.783779),
   (-73.927829, 40.781075)],
  'Buffer': 2,
  'Buffer_Coords': [(-73.94648356990037, 40.783453230873405),
   (-73.94120155505074, 40.79465514352185),
   (-73.94017105868987, 40.79599229449111),
   (-73.93708362837576, 40.80014686790213),
   (-73.9301053295897, 40.811942081274125),
   (-73.91304868398235, 40.80698687384361),
   (-73.90959455847604, 40.80487830831102),
   (-73.90395432040015, 40.79403505948594),
   (-73.90862632259017, 40.78481513758067),
   

In [137]:
# create Name/Coord dictionaries
exterior = {r[x]['Name']: r[x]['Exterior_Coords'] for x in r}
buffer = {r[x]['Name']: r[x]['Buffer_Coords'] for x in r}


In [138]:
# Create lists to use for developing geopandas dataframes
ex_n = [r[x]['Name'] for x in r]
ex_c = [Polygon(r[x]['Exterior_Coords']) for x in r]
bf_c = [Polygon(r[x]['Buffer_Coords']) for x in r]

In [139]:
# Create GeoPandas Dataframes and send to file for plotting
d = {'Zone': ex_n, 'geometry': ex_c}
e = {'Zone': ex_n, 'geometry': bf_c}
ex_gdf = geopandas.GeoDataFrame(d, crs="EPSG:4326")
bf_gdf = geopandas.GeoDataFrame(e, crs="EPSG:4326")
ex_gdf.to_file("exterior.geojson", driver="GeoJSON")
bf_gdf.to_file("buffer.geojson", driver="GeoJSON")
bf_gdf

/Users/paul.nisenson/.conda/envs/gis_project/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/Users/paul.nisenson/.conda/envs/gis_project/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


,Zone,geometry
0,East Harlem,"POLYGON ((-73.94648 40.78345, -73.94120 40.794..."
1,Gramercy Park,"POLYGON ((-74.00002 40.73263, -74.00138 40.748..."
2,Upper East Side,"POLYGON ((-73.97484 40.75500, -73.97925 40.772..."
3,Central Harlem,"POLYGON ((-73.97987 40.80303, -73.97764 40.818..."
4,Chelsea and Clinton,"POLYGON ((-74.02551 40.74852, -74.02788 40.758..."
5,Upper West Side,"POLYGON ((-74.00211 40.76763, -73.99437 40.790..."
6,Washington Heights,"POLYGON ((-73.96209 40.81051, -73.98654 40.818..."
7,Inwood,"POLYGON ((-73.94286 40.82631, -73.93631 40.851..."
8,Greenwich Village and Soho,"POLYGON ((-74.02783 40.71599, -74.02444 40.732..."
9,Lower Manhattan,"POLYGON ((-74.00319 40.69745, -73.98497 40.691..."


In [147]:
# create list in format for NYP
# format is [{latitude: "37.472", longitude: "-122.248"},{latitude: "37.525", longitude: "-122.163"},{latitude: "37.399", longitude: "-122.059"},{latitude: "37.472", longitude: "-122.248"}]
# i.e. list of dictionaries


def coord_list(coord_type):
    tester = []
    string = ''
    for y in coord_type:
        for x in coord_type[y]:
            string += (f'{{latitude: "{x[1]}", longitude: "{x[0]}"}},')
        tester.append([f'{y} : {string[0:-1]}'])
        string = ''
    return tester

pd.DataFrame({"Exterior": coord_list(exterior), "Buffer":coord_list(buffer)}).to_csv('Coordinates.csv')
pd.DataFrame({"Exterior": coord_list(exterior), "Buffer":coord_list(buffer)})


,Exterior,Buffer
0,"[East Harlem : {latitude: ""40.781184"", longitu...","[East Harlem : {latitude: ""40.783453230873405""..."
1,"[Gramercy Park : {latitude: ""40.736962"", longi...","[Gramercy Park : {latitude: ""40.73263485554648..."
2,"[Upper East Side : {latitude: ""40.758528"", lon...","[Upper East Side : {latitude: ""40.755001660557..."
3,"[Central Harlem : {latitude: ""40.797478"", long...","[Central Harlem : {latitude: ""40.8030307901594..."
4,"[Chelsea and Clinton : {latitude: ""40.740291"",...","[Chelsea and Clinton : {latitude: ""40.74851817..."
5,"[Upper West Side : {latitude: ""40.778168"", lon...","[Upper West Side : {latitude: ""40.767629344527..."
6,"[Washington Heights : {latitude: ""40.829724"", ...","[Washington Heights : {latitude: ""40.810506947..."
7,"[Inwood : {latitude: ""40.807955"", longitude: ""...","[Inwood : {latitude: ""40.82630899124321"", long..."
8,"[Greenwich Village and Soho : {latitude: ""40.7...","[Greenwich Village and Soho : {latitude: ""40.7..."
9,"[Lower Manhattan : {latitude: ""40.687934"", lon...","[Lower Manhattan : {latitude: ""40.697450143576..."
